In [13]:
# 0 Housekeeping. Clear variable space
########################################################################################################################
from IPython import get_ipython  # run magic commands

ipython = get_ipython()
ipython.magic("reset -f")
ipython = get_ipython()
ipython.magic("load_ext autoreload")
ipython.magic("autoreload 2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# 1 Import Libraries and Set Global Parameters
####################################################################################################
# 1.1 Import Python Libraries
############################################
from datetime import datetime
import os, sys, shutil
import pandas as pd
import geopandas as gpd
import pyarrow as pa
import pyarrow.parquet as pq

In [15]:
# 1.2 Set Global Parameters
############################################
if os.getlogin() == "WylieTimmerman":
    path_working = r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\OD\OneDrive - Foursquare ITP\Projects\WMATA_AVL")
    path_sp = r"C:\Users\WylieTimmerman\Documents\projects_local\wmata_avl_local"
    path_source_data = os.path.join(path_sp,"data","00-raw")
    path_processed_data = os.path.join(path_sp, "data","02-processed")
    path_segments = os.path.join(path_working,"data","02-processed")
elif os.getlogin() == "abibeka":
    path_working = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\Github\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(path_working)
    path_source_data = r"C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\WMATA-AVL\Data"
    path_processed_data = os.path.join(path_source_data, "ProcessedData")
    path_segments = path_processed_data
elif os.getlogin() == "E048374":
    # Working Paths
    path_working = r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL"
    os.chdir(os.path.join(path_working))
    sys.path.append(r"C:\Users\E048374\OneDrive - WMATA\rawnav_rachel_fork\WMATA_AVL")
    path_source_data = r"\\l-600730\RawNavArchive"
    path_sp = r"C:\Users\E048374\Documents\RawNav"
    path_processed_data = os.path.join(path_sp, "data", "02-processed")
    path_segments = os.path.join(path_working, "data", "02-processed")
    
else:
    raise FileNotFoundError("Define the path_working, path_source_data, gtfs_dir, \
                            ZippedFilesloc, and path_processed_data in a new elif block")

In [16]:
# Globals
q_jump_route_list = ['52']
analysis_routes = q_jump_route_list
# analysis_routes = ['S1']
# analysis_routes = ['S1', 'S9', 'H4', 'G8', '64']
# analysis_routes = ['S2','S4','H1','H2','H3','79','W47']
# took out Sunday bc doesn't exist in my test data (BAM)
analysis_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
# analysis_days = ['Wednesday','Thursday','Friday']

In [17]:
# EPSG code for WMATA-area work
wmata_crs = 2248
# 1.3 Import User-Defined Package
############################################
import wmatarawnav as wr

In [23]:
segments = (
    gpd.read_file(os.path.join(path_segments,"seg_5201_by_stop.geojson"), dtype={'pattern':'int32'})
    .to_crs(wmata_crs)
)[['seg_name_id', 'name_str', 'geoid', 'stop_id',
       'length', 'geometry']]

In [24]:
seg_pattern = pd.read_csv(os.path.join(path_segments,"stop_seq_pattern_5201_by_stop.csv"),
                         dtype={'route':str, 'PATTERN_ID':str, 'pattern':'int32'})
seg_pattern

,seg_name_id,geoid,stop_id,route,pattern
0,14th_25,7243,19143,52,1


In [25]:
wmata_schedule_dat = (
    pd.read_csv(
        os.path.join(path_sp, "wmata_schedule_data_q_jump_routes.csv"),
        index_col = 0
    )
    .reset_index(drop=True)
)

In [29]:
# 3 Merge Additional Geometry
####################################################################################################

# 3.1 Rawnav-Segment ########################

# Make Output Directory
path_seg_summary = os.path.join(path_processed_data,"by_stop", "segment_summary.parquet")
shutil.rmtree(path_seg_summary, ignore_errors=True) 
os.mkdir(path_seg_summary)

path_seg_index = os.path.join(path_processed_data,"by_stop", "segment_index.parquet")
shutil.rmtree(path_seg_index, ignore_errors=True) 
os.mkdir(path_seg_index)

In [30]:
# Iterate
for analysis_route in analysis_routes:
    print("*" * 100)
    print(f'Processing analysis route {analysis_route}')
    for analysis_day in analysis_days:
        print(f'Processing analysis route {analysis_route} for {analysis_day}...')
        
        # Reload data
        try:
            rawnav_dat = (
                wr.read_cleaned_rawnav(
                   analysis_routes_ = analysis_route,
                   analysis_days_ = analysis_day,
                   path = os.path.join(path_processed_data, "rawnav_data.parquet"))
                .drop(columns=['blank', 'lat_raw', 'long_raw', 'sat_cnt'])
                )
        except:
            print(f'No data on analysis route {analysis_route} for {analysis_day}')
            continue
        else:
   
            # Reload Data
            rawnav_summary_dat = (
                wr.read_cleaned_rawnav(
                    analysis_routes_ = analysis_route,
                    analysis_days_ = analysis_day,
                    path = os.path.join(path_processed_data, "rawnav_summary.parquet")
                )
            )

            # Subset Rawnav Data to Records Desired
            rawnav_summary_dat = rawnav_summary_dat.query('not (run_duration_from_sec < 600 | dist_odom_mi < 2)')
            
            rawnav_qjump_dat = rawnav_dat.merge(rawnav_summary_dat[['filename', 'index_run_start']], 
                                                on=['filename', 'index_run_start'],
                                                how='right')
            
            # Address Remaining Col Format issues
            rawnav_qjump_gdf = (
                gpd.GeoDataFrame(
                    rawnav_qjump_dat, 
                    geometry = gpd.points_from_xy(
                        rawnav_qjump_dat.long,
                        rawnav_qjump_dat.lat
                    ),
                    crs='EPSG:4326')
                .to_crs(epsg=wmata_crs)
            )
    
            # Iterate on over Pattern-Segments Combinations Applicable to Route
            xwalk_seg_pattern_subset = seg_pattern[['route','pattern','seg_name_id']].copy()
                        
            for seg in xwalk_seg_pattern_subset.seg_name_id.unique():
                print('Processing segment {} ...'.format(seg))

                # We pass the rawnav data and summary tables, check against a segment,
                # and use the patterns_by_seg to indicate which patterns should be examined
                index_run_segment_start_end, summary_run_segment = (
                    wr.merge_rawnav_segment(
                        rawnav_gdf_=rawnav_qjump_gdf,
                        rawnav_sum_dat_=rawnav_summary_dat,
                        target_=segments.loc[segments.seg_name_id == seg],
                        patterns_by_seg_=xwalk_seg_pattern_subset.loc[xwalk_seg_pattern_subset.seg_name_id == seg]
                    )
                )
                # Note that because seg_pattern_first_last is defined for route and pattern,
                # our summary will implicitly drop any runs that are on 'wrong' pattern(s) for 
                # a route. 
                
                index_run_segment_start_end['wday'] = analysis_day
                summary_run_segment['wday'] = analysis_day
                
                # The additional partitioning here is excessive, but if fits better in the 
                # iterative/chunking process above
                pq.write_to_dataset(
                    table = pa.Table.from_pandas(summary_run_segment),
                    root_path = path_seg_summary,
                    partition_cols = ['route','wday','seg_name_id']
                )
                
                pq.write_to_dataset(
                    table = pa.Table.from_pandas(index_run_segment_start_end),
                    root_path = path_seg_index,
                    partition_cols = ['route','wday','seg_name_id']
                )

****************************************************************************************************
Processing analysis route 52
Processing analysis route 52 for Monday...
Processing segment 14th_25 ...
Processing analysis route 52 for Tuesday...
Processing segment 14th_25 ...
Processing analysis route 52 for Wednesday...
Processing segment 14th_25 ...
Processing analysis route 52 for Thursday...
Processing segment 14th_25 ...
Processing analysis route 52 for Friday...
Processing segment 14th_25 ...
Processing analysis route 52 for Saturday...
Processing segment 14th_25 ...
